In [1]:

%pprint

Pretty printing has been turned OFF


In [39]:

from bs4.element import NavigableString
import random
import re

lt_regex = re.compile(r'\s+<')
gt_regex = re.compile(r'>\s+')

def clean_html_str(html_obj):
    html_str = str(html_obj)
    html_str = html_str.strip()
    html_str = lt_regex.sub('<', html_str)
    html_str = gt_regex.sub('>', html_str)
    
    return html_str

def get_navigable_children(tag, result_list=[]):
    if type(tag) is not NavigableString:
        for child_tag in tag.children:
            result_list = get_navigable_children(child_tag, result_list)
    else:
        base_str = clean_html_str(tag)
        if base_str:
            tag_str = clean_html_str(tag.parent)
            if tag_str.count('<') > 2:
                tag_str = base_str
            result_list.append(tag_str)
    
    return result_list

#div_tag = random.choice(div_tag_list)


----

In [259]:

import numpy as np

files_list = os.listdir(saves_html_folder)
file_name = random.choice(files_list)
file_path = os.path.join(saves_html_folder, file_name)
with open(file_path, 'r', encoding='utf-8') as f:
    html_str = f.read()
    job_soup = BeautifulSoup(html_str, 'lxml')
    body_soup = job_soup.find_all(name='body')[0]
    child_strs_list = get_navigable_children(body_soup, [])
    for child_str, pred_array in zip(child_strs_list, predict_percent_fit(child_str_list)):
        print()
        print(pred_array)
        print(child_str)


[0.5 0.5]
<p>Spokeo is a people search engine that both enlightens and empowers our customers. With over 12 billion records and 18 million visitors per month, we reconnect friends, reunite families, prevent fraud, and more.</p>

[0.99834671 0.00165329]
We are looking for a

[0.8738471 0.1261529]
<i>Data Scientist, Product Development</i>

[0.5 0.5]
to join us in Pasadena, CA, and help us turn our vast repository of data into usable information leveraging solutions that include matching techniques, machine learning, and graph theory to resolve entities across disparate data sets. Ultimately, these entity relationships will be used to build models that will be incorporated into the Spokeo product suite. This is part of Spokeo's Data Operations Team and is not an analytics-based role.

[0.84560529 0.15439471]
<b>Responsibilities:</b>

[0.91212559 0.08787441]
<p>Profiling, processing, and evaluating large structured and unstructured data sets.</p>

[0.98866173 0.01133827]
<p>Developing da

In [260]:

df = pd.DataFrame(child_strs_list, columns=['navigable_parent'])
df['is_header'] = False
id_list = []
df.loc[id_list, 'is_header'] = True
df.tail(60)

,navigable_parent,is_header
0,<p>Spokeo is a people search engine that both ...,False
1,We are looking for a,False
2,"<i>Data Scientist, Product Development</i>",False
3,"to join us in Pasadena, CA, and help us turn o...",False
4,<b>Responsibilities:</b>,False
5,"<p>Profiling, processing, and evaluating large...",False
6,<p>Developing data-driven models.</p>,False
7,<p>Data mining using state-of-the-art methods....,False
8,<p>Collaborating with product and engineering ...,False
9,<p>Productionalizing and maintaining algorithm...,False


In [256]:

try:
    for row_index, row_series in df.iterrows():
        navigable_parent = row_series.navigable_parent
        is_header = row_series.is_header
        child_str_dict[navigable_parent] = is_header
    s.store_objects(child_str_dict=child_str_dict)
    rows_list = [{'navigable_parent': navigable_parent, 'is_header': is_header} for navigable_parent, is_header in child_str_dict.items()]
    child_str_df = pd.DataFrame(rows_list)
    s.store_objects(child_str_df=child_str_df)
except Exception as e:
    print(str(e).strip())

Pickling to C:\Users\dev\Documents\repositories\notebooks\job-hunting\saves\pickle\child_str_dict.pickle
Pickling to C:\Users\dev\Documents\repositories\notebooks\job-hunting\saves\pickle\child_str_df.pickle


In [257]:

# Re-transform the bag-of-words and tf-idf from the new manual scores
child_str_dict = s.load_object('child_str_dict')
rows_list = [{'navigable_parent': navigable_parent, 'is_header': is_header} for navigable_parent, is_header in child_str_dict.items()]
child_str_df = pd.DataFrame(rows_list)
sents_list = child_str_df.navigable_parent.tolist()

# Bag-of-words
cv = CountVectorizer(lowercase=False, tokenizer=regex_tokenizer, ngram_range=(1, 3))
bow_matrix = cv.fit_transform(sents_list)
s.store_objects(cs_cv_vocab=cv.vocabulary_)

# Tf-idf must get from Bag-of-words first
tt = TfidfTransformer()
tfidf_matrix = tt.fit_transform(bow_matrix)
s.store_objects(cs_tt=tt)

# Re-train the classifier
X = tfidf_matrix.toarray()
y = child_str_df.is_header.to_numpy()
fit_estimators_dict = s.load_object('fit_estimators_dict')
child_str_clf = fit_estimators_dict['LogisticRegression']
child_str_clf.fit(X, y)
fit_estimators_dict['LogisticRegression'] = child_str_clf
s.store_objects(child_str_clf=child_str_clf, fit_estimators_dict=fit_estimators_dict)

# Re-calibrate the inference engine
cs_cv_vocab = s.load_object('cs_cv_vocab')
cs_cv = CountVectorizer(vocabulary=cs_cv_vocab)
cs_cv._validate_vocabulary()
cs_tt = s.load_object('cs_tt')
def predict_percent_fit(navigable_parents_list):
    y_predict_proba = np.array([])
    if len(navigable_parents_list):
        X_test = cs_tt.transform(cs_cv.transform(navigable_parents_list)).toarray()
        y_predict_proba = child_str_clf.predict_proba(X_test)
    
    return y_predict_proba
print('Retraining complete')

Pickling to C:\Users\dev\Documents\repositories\notebooks\job-hunting\saves\pickle\cs_cv_vocab.pickle
Pickling to C:\Users\dev\Documents\repositories\notebooks\job-hunting\saves\pickle\cs_tt.pickle
Pickling to C:\Users\dev\Documents\repositories\notebooks\job-hunting\saves\pickle\child_str_clf.pickle
Pickling to C:\Users\dev\Documents\repositories\notebooks\job-hunting\saves\pickle\fit_estimators_dict.pickle
Retraining complete


In [258]:

sample_list = random.sample([(w, i) for w, i in cs_cv_vocab.items()], 20)
sorted([(w, round(cs_tt.idf_[i], 2)) for w, i in sample_list], key=lambda x: x[1], reverse=False)

[('Locations:', 7.12), ('the vacancy closing', 7.52), ('MINIMUM', 7.52), ('homeland security civil', 7.52), ('of the Air', 7.52), ('are effective', 7.52), ('Amazon Sagemaker </li>', 7.52), ('effectively with other', 7.52), ('re engineer', 7.52), ('resilience', 7.52), ('candidate must demonstrate', 7.52), ('with attention', 7.52), ('<p> At FireEye', 7.52), ('the constant improvement', 7.52), ('plants oxy', 7.52), ('parallel or distributed', 7.52), ('50 DOES', 7.52), ('will meaningfully', 7.52), ('unsolicited resumes', 7.52), ('for high', 7.52)]


----

In [32]:

%run ../../load_magic/storage.py
s = Storage()
try:
    child_str_dict = s.load_object('child_str_dict')
    rows_list = [{'navigable_parent': navigable_parent, 'is_header': is_header} for navigable_parent, is_header in child_str_dict.items()]
    child_str_df = pd.DataFrame(rows_list)
    s.store_objects(child_str_df=child_str_df)
except:
    child_str_df = s.load_object('child_str_df')
    try:
        child_str_df = pd.concat([child_str_df, df])
    except:
        pass
    child_str_dict = child_str_df.set_index('navigable_parent').to_dict()['is_header']
    s.store_objects(child_str_dict=child_str_dict)

Pickling to C:\Users\dev\Documents\repositories\notebooks\job-hunting\saves\pickle\child_str_df.pickle


In [153]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression

scanner_regex = re.compile(r'(</?[^><]+>|\w+:?)')
try:
    fit_estimators_dict = s.load_object('fit_estimators_dict')
except:
    fit_estimators_dict = {}
    fit_estimators_dict['LogisticRegression'] = LogisticRegression(**{'C': 7.5, 'class_weight': 'balanced', 'dual': True, 'fit_intercept': False,
                                                                      'max_iter': 64, 'penalty': 'l2', 'solver': 'liblinear', 'tol': 1e-09})
    s.store_objects(fit_estimators_dict=fit_estimators_dict)
def regex_tokenizer(corpus):
    
    return [match.group() for match in re.finditer(scanner_regex, corpus)]


----
# Download Job HTML

In [29]:

from IPython.display import clear_output
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs

%run ../../load_magic/storage.py
s = Storage()
base_url = 'https://www.indeed.com'
site_url = base_url + '/jobs'
site_url = '?'.join([site_url, 'q=data+scientist'])
site_url = '&'.join([site_url, 'jt=fulltime'])
site_url = '&'.join([site_url, 'remotejob=032b3046-06a3-4876-8dfd-474eb5e7ed11'])
site_url = '&'.join([site_url, 'vjk=ca16b63c03e40c57'])
#site_url = '&'.join([site_url, 'pp=gQAPAAABdY7RMKwAAAABkQdgZAAkAQBEKPpaoZstIag3f-UtQXXG_HFSo1gfBp9OQ0B0TvZ4yMp4AAA'])
start_num = 0
job_urls_list = s.load_object('job_urls_list')
space_regex = re.compile(r'[\s<>:"/\\\|\?\*_]+')
saves_html_folder = os.path.join(s.saves_folder, 'html')
print_regex = re.compile(r'[\x9c-\x9d\uf0b7\u200b\ufb02]+')
fccid_htmls_list = [fn.split('_')[-1] for fn in os.listdir(saves_html_folder)]
s.encoding_type = ['latin1', 'iso8859-1', 'utf-8'][2]

In [28]:

for job_url in job_urls_list:
    qs = urlparse(job_url).query
    query_dict = parse_qs(qs)
    fccid_str = query_dict['fccid'][0]
    file_name = f'{fccid_str}.html'
    if file_name not in fccid_htmls_list:
        job_page = requests.get(url=job_url)
        job_soup = BeautifulSoup(job_page.content, 'lxml')
        title_str = job_soup.find_all(name='title')[0].text.strip()
        if 'CAPTCHA' in title_str:
            print(title_str)
            break
        file_name = space_regex.sub('_', title_str)
        file_name = f'{file_name}_{fccid_str}.html'
        file_path = os.path.join(saves_html_folder, file_name)
        body_soup = job_soup.find_all(name='body')[0]
        html_str = '<html><head><title>' + title_str + '</title></head><body>'
        with open(file_path, 'w', encoding='utf-8') as f:
            print(html_str, file=f)
            for div_tag in body_soup.find_all(name='div', class_='jobsearch-JobComponent-description'):
                for s in div_tag.select('template'):
                    s.extract()
                for s in div_tag.select('script'):
                    s.extract()
                div_str = div_tag.prettify(formatter='html')
                div_str = print_regex.sub('', div_str)
                print(div_str, file=f)
            print('</body></html>', file=f)
print('Complete.')

reCAPTCHA solve page
Complete.


In [26]:

position = 4984
error_str = str(div_str[position-1:position+2])
print(f'"{error_str}"')
[str(x) for x in bytearray(error_str, 'utf-8', 'replace')]

"t, "


['116', '44', '32']

In [30]:

len(fccid_htmls_list)

446

In [3]:

while True:
    page_url = '&'.join([site_url, f'start={start_num}'])
    start_num += 10
    site_page = requests.get(url=page_url)
    page_soup = BeautifulSoup(site_page.content, 'lxml')
    row_div_list = page_soup.find_all(name='div', class_=['row', 'result'])
    row_count = len(row_div_list)
    if row_count == 0:
        print('Nothing left')
        break
    for i, row_div in enumerate(row_div_list):
        link = row_div.find_all(name='a')[0]
        if 'title' in link.attrs:
            if 'href' in link.attrs:
                job_url = base_url + link['href']
                qs = urlparse(job_url).query
                query_dict = parse_qs(qs)
                if 'fccid' in query_dict:
                    job_urls_list.append(job_url)
                    s.store_objects(verbose=False, job_urls_list=job_urls_list)
                    clear_output(wait=True)
                    print(f'{page_url}')
                    print(f'{i}/{row_count}: {job_url}')

https://www.indeed.com/jobs?q=data+scientist&jt=fulltime&remotejob=032b3046-06a3-4876-8dfd-474eb5e7ed11&vjk=ca16b63c03e40c57&start=4670
14/15: https://www.indeed.com/rc/clk?jk=3c08e53cedd536b6&fccid=dae40dde876a8942&vjs=3
Nothing left


In [4]:

page_url = '&'.join([site_url, f'start={start_num}'])
start_num += 10
site_page = requests.get(url=page_url)
site_page.content

b'<html>\n<head>\n<title>reCAPTCHA solve page</title>\n<script src="https://www.google.com/recaptcha/api.js" async defer></script>\n</head>\n<body>\n<form action="/jobs?q=data+scientist&jt=fulltime&remotejob=032b3046-06a3-4876-8dfd-474eb5e7ed11&vjk=ca16b63c03e40c57&start=4690" method="POST">\n<div class="g-recaptcha" data-sitekey="6Lc5uUEUAAAAAHBFgzyc9no20EC0e7A-_R0QFgww"></div>\n<br/>\n<input type="submit" value="Submit">\n</form>\n</body>\n</html>'

In [14]:

job_url

'https://www.indeed.com/rc/clk?jk=b6506065408f4f15&fccid=0ce02a3e84d031c6&vjs=3'

In [2]:

import os

from selenium import webdriver
from base64 import b64encode
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import selenium.webdriver.support.ui as ui

In [3]:

#siteUrl = 'https://www.indeed.com/jobs?q=data%20scientist&jt=fulltime&remotejob=032b3046-06a3-4876-8dfd-474eb5e7ed11&vjk=ca16b63c03e40c57'
base_url = 'https://www.indeed.com'
site_url = base_url + '/jobs'
site_url = '?'.join([site_url, 'q=data%20scientist'])
site_url = '&'.join([site_url, 'jt=fulltime'])
site_url = '&'.join([site_url, 'remotejob=032b3046-06a3-4876-8dfd-474eb5e7ed11'])
site_url = '&'.join([site_url, 'vjk=ca16b63c03e40c57'])
print(site_url)

https://www.indeed.com/jobs?q=data%20scientist&jt=fulltime&remotejob=032b3046-06a3-4876-8dfd-474eb5e7ed11&vjk=ca16b63c03e40c57


In [4]:

# Retrieve the page with tag results and set it up to be scraped
sitePage = requests.get(url=site_url)
sitePageSoup = BeautifulSoup(sitePage.content, 'lxml')

In [5]:

saves_txt_folder = os.path.join(s.saves_folder, 'txt')
os.makedirs(name=saves_txt_folder, exist_ok=True)

In [6]:

row_div_list = sitePageSoup.find_all(name='div', class_=['row', 'result'])
reqs_regex = re.compile('ducat|xperience|equire')
html_regex = re.compile('<[^>]+>')

In [36]:

div_tag_list = []
for row_div in row_div_list:
    link = row_div.find_all(name='a')[0]
    if 'title' in link.attrs:
        if 'href' in link.attrs:
            job_url = base_url + link['href']
            qs = urlparse(job_url).query
            query_dict = parse_qs(qs)
            if 'fccid' in query_dict:
                title_str = space_regex.sub('_', link['title'].strip())
                fccid_str = query_dict['fccid'][0]
                file_name = f'{title_str}_{fccid_str}.html'
                file_path = os.path.join(saves_html_folder, file_name)
                job_page = requests.get(url=job_url)
                job_soup = BeautifulSoup(job_page.content, 'lxml')
                body_soup = job_soup.find_all(name='body')[0]
                html_str = '<html><head><title>' + title_str.replace('_', ' ') + '</title></head><body>'
                with open(file_path, 'w', encoding='utf-8') as f:
                    print(html_str, file=f)
                    for div_tag in body_soup.find_all(name='div', class_='jobsearch-JobComponent-description'):
                        div_tag_list.append(div_tag)
                        for s in div_tag.select('template'):
                            s.extract()
                        for s in div_tag.select('script'):
                            s.extract()
                        div_str = div_tag.prettify(formatter='html')
                        div_str = print_regex.sub('', div_str)
                        print(div_str, file=f)
                    print('</body></html>', file=f)
print('Complete.')

Complete.


In [201]:

df.loc[0, 'navigable_parent']

'<div class="jobsearch-JobDescriptionSection-jobDescriptionTitle icl-u-xs-my--md" id="jobDescriptionTitle">Full Job Description</div>'

In [214]:

import pandas as pd

div_tag = div_tag_list[11]
child_str_list = get_navigable_children(div_tag, [])
df = pd.DataFrame(child_str_list, columns=['navigable_parent'])
df['is_header'] = False
#df.loc[[0, 2, 4, 17, 19], 'is_header'] = True
df

IndexError: list index out of range

In [ ]:

text_list = ['What we are looking for', 'Key skills and Experience', 'Minimum qualifications', 'The Essentials', 'Qualifications',
             'Skills and experience', 'Required Qualifications', 'What your background looks like', "We(?:&rsquo;|')re looking for \w+ who have",
             'Experience/Minimum Qualifications', 'Requirements', 'We are looking for someone with', 'Qualifications']
reqs_regex = re.compile(f'^\\s*({'|'.join(text_list)}):?\\s*$', re.IGNORECASE | re.MULTILINE)
for match_obj in reqs_regex.finditer(text_str):
    # match start: match_obj.start()
    # match end (exclusive): match_obj.end()
    # matched text: match_obj.group()

In [39]:

[f'div_tag.{fn}' for fn in dir(div_tag) if 'child' in fn.lower()]

['div_tag._lastRecursiveChild', 'div_tag.childGenerator', 'div_tag.children', 'div_tag.findChild', 'div_tag.findChildren', 'div_tag.recursiveChildGenerator', 'div_tag.replaceWithChildren', 'div_tag.replace_with_children']

In [117]:

selector_list = ['body', 'div', 'div.jobsearch-ViewJobLayout-fluidContainer',
                 'div.jobsearch-ViewJobLayout-content.jobsearch-ViewJobLayout-mainContent', 'div', 'div',
                 'div.jobsearch-ViewJobLayout-jobDisplay', 'div.jobsearch-JobComponent', 'div.jobsearch-JobComponent-description']
content_selector = ' > '.join(selector_list)

def has_role_attr(tag):
    
    return tag.has_attr('role')

In [123]:

for row_div in row_div_list:
    link = row_div.find_all(name='a')[0]
    if 'title' in link.attrs:
        if 'href' in link.attrs:
            job_url = base_url + link['href']
            qs = urlparse(job_url).query
            query_dict = parse_qs(qs)
            if 'fccid' in query_dict:
                title_str = space_regex.sub('_', link['title'].strip())
                fccid_str = query_dict['fccid'][0]
                file_name = f'{title_str}_{fccid_str}.txt'
                file_path = os.path.join(saves_txt_folder, file_name)
                job_page = requests.get(url=job_url)
                job_soup = BeautifulSoup(job_page.content, 'lxml')
                body_soup = job_soup.find_all(name='body')[0]
                with open(file_path, 'w', encoding='utf-8') as f:
                    print(title_str.replace('_', ' '), file=f)
                    print('-'*100, file=f)
                    for div_tag in body_soup.find_all(name='div', class_='jobsearch-JobComponent-description'):
                        for s in div_tag.select('template'):
                            s.extract()
                        for s in div_tag.select('script'):
                            s.extract()
                        print(file=f)
                        for div_str in div_tag.strings:
                            div_str = print_regex.sub('', div_str).strip().replace('\x92', "'")
                            print(div_str, file=f)
print('Complete.')

Complete.


In [106]:

for row_div in row_div_list:
    link = row_div.find_all(name='a')[0]
    if 'title' in link.attrs:
        if 'href' in link.attrs:
            job_url = base_url + link['href']
            qs = urlparse(job_url).query
            query_dict = parse_qs(qs)
            if 'fccid' in query_dict:
                title_str = space_regex.sub('_', link['title'].strip())
                fccid_str = query_dict['fccid'][0]
                file_name = f'{title_str}_{fccid_str}.txt'
                file_path = os.path.join(saves_txt_folder, file_name)
                with open(file_path, 'w', encoding='utf-8') as f:
                    print(title_str.replace('_', ' '), file=f)
                    print('-'*100, file=f)
                    job_page = requests.get(url=job_url)
                    job_soup = BeautifulSoup(job_page.content, 'lxml')
                    div_str_set = set()
                    for div_tag in job_soup.find_all(name='body')[0].find_all(name='div'):
                        for s in div_tag.select('template'):
                            s.extract()
                        for s in div_tag.select('script'):
                            s.extract()
                        print(file=f)
                        for div_str in div_tag.strings:
                            div_str = print_regex.sub('', div_str).strip().replace('\x92', "'")
                            #if len(div_str) > 32:
                            if div_str not in div_str_set:
                                print(div_str, file=f)
                                div_str_set.add(div_str)
print('Complete.')

Complete.


In [71]:

for div_tag in job_soup.find_all(name='body')[0].find_all(name='div'):
    for s in div_tag.select('script'):
        s.extract()
    for s in div_tag.select('template'):
        s.extract()
    print()
    for div_str in div_tag.strings:
        if len(div_str) > 32:
            print(f'{div_str}')
    break


Read what people are saying about working here.

Do you enjoy testing the limits of possibility? At McGraw Hill, our Data Scientists drive progress and help build the future of learning. If you have the passion and technical expertise to thrive in an innovative and agile environment, we want to learn more about you.

McGraw Hill, the leading provider of digital and print educational materials is looking for a Data Scientist Intern to work remotely within our Digital Platform Group (DPG). This group is responsible for building and supporting innovative on-line platforms to power learning across our K-12, Higher Education, International, and Professional segments. As part of this team, the Analytics and Reporting department is building best-in-class applications which leverage data and machine learning for advanced analytics and adaptive learning products. If you are interested in contributing to the future of digital and remote learning, join us!

This role works hands-on within our Da

In [34]:
    if len(a_list)==0:
        print(row_div)
        print()
        print('-'*100)
        print()

In [20]:

def has_class_and_id(tag):
    
    return tag.has_attr('class') and tag.has_attr('id')

# <span class="summary">We need a <b>data</b> <b>scientist</b> and <b>data</b> wrangler. Maybe survey <b>data</b>. 
# Better still if you have some demonstrable experience with more advanced machine learning methods...</span>
summary_list = sitePageSoup.find_all(name='span', class_='summary')
for summary in summary_list:
    print(summary.text.strip())
    print()

Accountabilities Lead the recruitment and allocation of appropriately skilled data scientists/health informatics to projects to ensure delivery of value...

Degree (in Engineering, Computer Science, Mathematics, or other scientific field of study) and 0-1 years of relevant experience, a Master’s degree (scientific...

He/she will be expected to lead experimental design, interpret data, troubleshoot experiments, coordinate junior scientists, and present work at both internal...

Integration of business data into automation workflows and scientific data into. Familiarity with scientific modeling and data preferred....

Knowledge of data science, data mining, machine learning and statistics. Knowledge of R and R plotting and/or Python matplotlib, Scikit, Matlab....

We are passionate about deep learning, reinforcement learning, semi-supervised and unsupervised learning, but are also considering other areas of machine...

Research Scientist - Computer Vision / Machine Learning. And depth i

In [3]:

# <a target="_blank" id="sja5" data-tn-element="jobTitle" class="jobtitle turnstileLink" 
# href="https://www.indeed.com/pagead/clk?mo=r&amp;ad=-...-...-...-...-...-...-...&
# amp;p=5&amp;sk=&amp;fvj=1&amp;tk=1c06s8995av53coj&amp;jsa=6565" 
# title="Perception Scientist for Marine Autonomy" rel="noopener nofollow" 
# onmousedown="sjomd('sja5'); clk('sja5');" onclick="setRefineByCookie([]); sjoc('sja5',0); convCtr('SJ')"
# >Perception <b>Scientist</b> for Marine Autonomy</a>
siteCss = '#sja5'
siteLinks = sitePageSoup.select(siteCss)
print(siteLinks)

max_page = 0
if len(siteLinks):
    max_page = int(siteLinks[0]["href"].split('/')[-1].split('-')[1].split(',')[0])
print(max_page)

WebDriverException: Message: connection refused


In [6]:
#<h1 class="srp-header">25,589 Used Vehicles for sale</h1>

for parser in ['lxml', 'html5lib', 'html.parser']:
    sitePageSoup = BeautifulSoup(page_html, parser)
    print(parser, len(sitePageSoup.select('h1.srp-header')))

#max_page = int(sitePageSoup.find_all("a", class_="js-last-page")[0].text)
#print(max_page)

lxml 1
html5lib 0
html.parser 1


In [10]:
sitePageSoup = BeautifulSoup(page_html, 'lxml')
srp_header = sitePageSoup.select('h1.srp-header')
if len(srp_header):
    max_record_string = srp_header[0].get_text().strip()
    print(max_record_string)
    max_record = int(re.sub('\\D+', '', max_record_string))
    print(max_record)
    max_page = round(max_record/50)
    print(max_page)

25,592 Used Vehicles for sale
25592
512


In [25]:
from random import randint
import urllib.parse

i = randint(1, max_page)
tagPageUrl = siteUrl+"&page="+str(i)
print(tagPageUrl)

# Retrieve the page with tag results and set it up to be scraped
tagPage = requests.get(url=tagPageUrl, proxies=proxies)
tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

links = tagPageSoup.find_all("a", class_="listing-row__photo")
link = links[randint(0,len(links)-1)]
url = urllib.parse.urljoin(tagPageUrl, re.sub(' ', '%20', link['href']))
print(url)

https://www.cars.com/for-sale/searchresults.action/?perPage=50&rd=99999&searchSource=UTILITY&sf1Dir=DESC&sf1Nm=price&sf2Dir=ASC&sf2Nm=miles&slrTypeId=28879&sortFeatures=buryLowPriceOlderThanSix&sortFeatures=buryNewLowPrice&sortFeatures=buryNoPrice&sortFeatures=buryUsedLowMileage&sortFeatures=buryUsedLowPrice&stkTypId=28881&yrId=20141&yrId=20142&yrId=20143&yrId=20144&yrId=20145&yrId=20180&yrId=20197&yrId=20198&yrId=20199&yrId=20200&yrId=20201&yrId=27381&yrId=34923&yrId=39723&yrId=47272&yrId=51683&yrId=56007&yrId=58487&zc=60606&page=31
https://www.cars.com/vehicledetail/detail/692753718/overview/


In [24]:
import re

page = requests.get(url=url, proxies=proxies)
soup = BeautifulSoup(page.content, 'lxml')

dealer_css = 'h4.dealer-name.cui-delta > span'
dealer_link_text = "Private Seller"
dealer_link = soup.select(dealer_css)
is_dealer = False
if len(dealer_link):
    match = re.search(dealer_link_text, dealer_link[0].get_text().strip())
    if match:
        is_dealer = False
    else:
        is_dealer = True
else:
    is_dealer = True
if not is_dealer:

    model = soup.find_all("h1", class_="vdp-header__title")
    if(len(model)):
        model = model[0].get_text().strip()
    else:
        model = "Unknown"
    print(model)

    price = soup.find_all("strong", class_='vdp-header__price--primary')
    if(len(price)):
        price = price[0].get_text().strip()
    else:
        price = "Unknown"
    print(price)

    phone_css = 'span.phone > span > span'
    phone = soup.select(phone_css)
    if(len(phone)):
        phone = phone[0].get_text().strip()
    else:
        phone = ""
    #print(phone)
    if(len(phone)):
        bigFile = open('cars.txt', 'a')
        bigFile.write(model + '\t' + price + '\t' + phone + '\n')
        bigFile.close()

Private Seller
Used 2008 Ford F250 Lariat Crew Cab Super Duty
$39,000
(908) 878-3257


In [6]:
for i in range(1, max_page+1):
    tagPageUrl = siteUrl+"&page="+str(i)

    # Retrieve the page with tag results and set it up to be scraped
    tagPage = requests.get(url=tagPageUrl, proxies=proxies)
    tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

    links = tagPageSoup.find_all("a", class_="listing-row__photo")
    for link in links:
        url = urllib.parse.urljoin(tagPageUrl, re.sub(' ', '%20', link['href']))
        page = requests.get(url=url, proxies=proxies)
        soup = BeautifulSoup(page.content, 'lxml')
        
        dealer_css = 'h4.dealer-name.cui-delta > span'
        dealer_link_text = "Private Seller"
        dealer_link = soup.select(dealer_css)
        is_dealer = False
        if len(dealer_link):
            match = re.search(dealer_link_text, dealer_link[0].get_text().strip())
            if match:
                is_dealer = False
            else:
                is_dealer = True
        else:
            is_dealer = True
        if not is_dealer:

            model = soup.find_all("h1", class_="vdp-header__title")
            if(len(model)):
                model = model[0].get_text().strip()
            else:
                model = "Unknown"
            #print(model)

            price = soup.find_all("strong", class_='vdp-header__price--primary')
            if(len(price)):
                price = price[0].get_text().strip()
            else:
                price = "Unknown"
            #print(price)

            phone_css = 'span.phone > span > span'
            phone = soup.select(phone_css)
            if(len(phone)):
                phone = phone[0].get_text().strip()
            else:
                phone = ""
            #print(phone)
            if(len(phone)):
                bigFile = open('cars.txt', 'a')
                bigFile.write(model + '\t' + price + '\t' + phone + '\n')
                bigFile.close()

In [3]:
# Retrieve the page with tag results and set it up to be scraped
tagPage = requests.get(url=tagPageUrl, proxies=proxies)
tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

In [25]:
html_doc = """
<cars-shop-srp-pagination><cui-pagination page="page" per-page="perPage" number-of-pages="numberOfPages" results-per-page-options="resultsPerPageOptions"><cui-page-button><a class="button previous-page" ng-click="prevPage($event)" ng-disabled="page === 1" rel="prev" ng-attr-data-linkname="{{dtmLinkPrev}}" data-linkname="" disabled="disabled">Prev</a> <a class="button next-page" ng-click="nextPage($event)" ng-disabled="page === numberOfPages" rel="next" ng-attr-data-linkname="{{dtmLinkNext}}" data-linkname="">Next</a> </cui-page-button> <cui-page-list><ul class="page-list"> <!----> <!----> <!----><li ng-repeat="p in pageList track by $index"> <a class="js-page-1 current-page" ng-click="goToPage($event, p)" ng-class="{'current-page': p === page}" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" data-linkname="">1</a> </li><!----><li ng-repeat="p in pageList track by $index"> <a class="js-page-2" ng-click="goToPage($event, p)" ng-class="{'current-page': p === page}" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" data-linkname="">2</a> </li><!----><li ng-repeat="p in pageList track by $index"> <a class="js-page-3" ng-click="goToPage($event, p)" ng-class="{'current-page': p === page}" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" data-linkname="">3</a> </li><!----><li ng-repeat="p in pageList track by $index"> <a class="js-page-4" ng-click="goToPage($event, p)" ng-class="{'current-page': p === page}" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" data-linkname="">4</a> </li><!----><li ng-repeat="p in pageList track by $index"> <a class="js-page-5" ng-click="goToPage($event, p)" ng-class="{'current-page': p === page}" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" data-linkname="">5</a> </li><!----> <!----><li ng-if="showLast">...</li><!----> <!----><li ng-if="showLast"> <a class="js-last-page" ng-click="goToPage($event, numberOfPages)">50</a> </li><!----> </ul> </cui-page-list> <cui-page-result-count><div class="select rpp"> <select ng-model="rppSelectModel.perPage" ng-model-options="{updateOn: 'default change', debounce: {default: 250, change: 0}}" ng-change="rppChange()" ng-options="rpp.text for rpp in rppSelectModel.options track by rpp.value" ng-attr-data-linkname="{{:: dtmListing}}" class="ng-pristine ng-untouched ng-valid ng-not-empty" data-linkname=""><option label="10 Per Page" value="10">10 Per Page</option><option label="20 Per Page" value="20">20 Per Page</option><option label="30 Per Page" value="30">30 Per Page</option><option label="50 Per Page" value="50" selected="selected">50 Per Page</option><option label="100 Per Page" value="100">100 Per Page</option></select> </div> </cui-page-result-count> </cui-pagination> </cars-shop-srp-pagination>
"""

soup2 = BeautifulSoup(html_doc, 'html.parser')

print(soup2.prettify())

<cars-shop-srp-pagination>
 <cui-pagination number-of-pages="numberOfPages" page="page" per-page="perPage" results-per-page-options="resultsPerPageOptions">
  <cui-page-button>
   <a class="button previous-page" data-linkname="" disabled="disabled" ng-attr-data-linkname="{{dtmLinkPrev}}" ng-click="prevPage($event)" ng-disabled="page === 1" rel="prev">
    Prev
   </a>
   <a class="button next-page" data-linkname="" ng-attr-data-linkname="{{dtmLinkNext}}" ng-click="nextPage($event)" ng-disabled="page === numberOfPages" rel="next">
    Next
   </a>
  </cui-page-button>
  <cui-page-list>
   <ul class="page-list">
    <!-- -->
    <!-- -->
    <!-- -->
    <li ng-repeat="p in pageList track by $index">
     <a class="js-page-1 current-page" data-linkname="" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" ng-class="{'current-page': p === page}" ng-click="goToPage($event, p)">
      1
     </a>
    </li>
    <!-- -->
    <li ng-repeat="p in pageList track by $index">

In [26]:
match = re.search(r'<a[^>]+class="js-last-page"[^>]+>([^><]+)</a>', str(soup2))
if match:
    max_page_string = match.group(1)
else:
    max_page_string = ""
if len(max_page_string):
    print(max_page_string)
    max_page = int(re.sub('\D+', '', max_page_string))
    print(max_page)

50
50


In [34]:
soup2.select("a")

[<a class="button previous-page" data-linkname="" disabled="disabled" ng-attr-data-linkname="{{dtmLinkPrev}}" ng-click="prevPage($event)" ng-disabled="page === 1" rel="prev">Prev</a>,
 <a class="button next-page" data-linkname="" ng-attr-data-linkname="{{dtmLinkNext}}" ng-click="nextPage($event)" ng-disabled="page === numberOfPages" rel="next">Next</a>,
 <a class="js-page-1 current-page" data-linkname="" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" ng-class="{'current-page': p === page}" ng-click="goToPage($event, p)">1</a>,
 <a class="js-page-2" data-linkname="" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" ng-class="{'current-page': p === page}" ng-click="goToPage($event, p)">2</a>,
 <a class="js-page-3" data-linkname="" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" ng-class="{'current-page': p === page}" ng-click="goToPage($event, p)">3</a>,
 <a class="js-page-4" data-linkname="" ng-attr-data-linkname=

In [33]:
url3 = "https://www.cars.com/for-sale/searchresults.action/?zc=60606&rd=99999&slrTypeId=28879&yrId=58487&yrId=56007&yrId=51683&yrId=47272&yrId=39723&yrId=34923&yrId=27381&yrId=20201&yrId=20145&yrId=20200&yrId=20144&yrId=20199&yrId=20143&yrId=20198&yrId=20142&yrId=20197&yrId=20141&yrId=20180&stkTypId=28881&sf1Nm=price&sf1Dir=DESC&sf2Nm=miles&sf2Dir=ASC&perPage=50&sortFeatures=buryUsedLowPrice&sortFeatures=buryNewLowPrice&sortFeatures=buryLowPriceOlderThanSix&sortFeatures=buryNoPrice&sortFeatures=buryUsedLowMileage&searchSource=GN_BREADCRUMB"
page3 = requests.get(url=url3, proxies=proxies)
soup3 = BeautifulSoup(page3.content, 'lxml')
soup3.find_all("a")

[<a class="logo" data-linkname="header-home" href="/" name="&amp;lid=header-home" ng-class="{'simple': smallLogo}" target="_self">
 <svg class="cars-logo" enable-background="new 0 0 165 70" height="70px" version="1.1" viewbox="0 0 165 70" width="165px" x="0px" xml:space="preserve" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" y="0px"><g id="Layer_1"><radialgradient cx="41.1323" cy="-3.0356" gradienttransform="matrix(-0.8303 0.5574 -0.4031 -0.6005 74.0593 -27.7844)" gradientunits="userSpaceOnUse" id="SVGID_1_" r="116.9038"><stop offset="0.3086" style="stop-color:#DEDEDE"></stop><stop offset="0.392" style="stop-color:#9E9E9E"></stop><stop offset="0.522" style="stop-color:#3F3F3F"></stop><stop offset="0.5807" style="stop-color:#1A1A1A"></stop><stop offset="0.6089" style="stop-color:#202020"></stop><stop offset="0.8339" style="stop-color:#4D4D4D"></stop></radialgradient><path d="M15.588,31.615c0-13.049,17.466-24.389,43.138-30.146C24.623,5.791-0.233,19.167-0.